<img src="ubiops_logo.svg" width="80">

# Quickstart
Deploy a simple float-to-float deployment in UbiOps using the python client library.


The first step is to [download](https://storage.googleapis.com/ubiops/example-deployment-packages/example_deployment_package.zip) a prepared deployment package (the deployment code).

In [ ]:
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/example_deployment_package.zip -o example_deployment_package.zip

Add your API token and project name. You can also adapt the deployment name and deployment version name or leave the default values. Afterwards we initialize the client library, which establishes the connection with UbiOps.

In [ ]:
API_TOKEN = 'Token <YOUR_API_KEY>'
PROJECT_NAME = '<YOUR_PROJECT_NAME>'
DEPLOYMENT_NAME = 'simple-tutorial'
DEPLOYMENT_VERSION = 'v1'

In [ ]:
import ubiops
configuration = ubiops.Configuration(host="https://api.ubiops.com/v2.1")
configuration.api_key['Authorization'] = API_TOKEN

Here we open the connection with the UbiOps API Client.

In [ ]:
client = ubiops.ApiClient(configuration)
api = ubiops.CoreApi(client)
api.service_status()

## Deploy

### Create a deployment

In [ ]:
deployment_template = ubiops.DeploymentCreate(
    name=DEPLOYMENT_NAME,
    description='A simple deployment that multiplies the input float by a random number.',
    input_type='structured',
    output_type='structured',
    input_fields=[{'name': 'input', 'data_type': 'double'}],
    output_fields=[{'name': 'output', 'data_type': 'double'}],
    labels={}
)

deployment = api.deployments_create(project_name=PROJECT_NAME, data=deployment_template)
print(deployment)

### Create a deployment version

<div class="alert alert-block alert-info">
    
<b>Deployment Mode:</b> The <i>deployment_mode</i> of a deployment version determines what kind of requests will be made to it. For <code>deployment_mode='express'</code>, direct (<i>synchronous</i>) requests will be made, resulting in <i>low latency</i> and a maximum duration of 1 hour for a single request. For <code>deployment_mode='batch'</code>, batch (<i>asynchronous</i>) requests will be made, resulting in a higher latency but a maximum duration of <i>48 hours</i> for a single request. You will retrieve a request id with which you can obtain the results later (see <i>request retention mode</i>).
<br/><br/>
<b>Request Retention Mode:</b>
Use <code>request_retention_mode='metadata'</code> or <code>request_retention_mode='full'</code> to store all direct and batch requests. <u>Only then you will be able to retrieve them later</u>. In this demo, we don't need to store them.

</div>

In [ ]:
version_template = ubiops.DeploymentVersionCreate(
    version=DEPLOYMENT_VERSION,
    language='python3.7',
    memory_allocation=256,
    maximum_instances=1,
    minimum_instances=0,
    maximum_idle_time=1800, # = 30 minutes
    deployment_mode='express',  # 'express' or 'batch'
    request_retention_mode='none'  # We don't need to store the requests in this demo
)

version = api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=version_template
)
print(version)

Upload the prepared deployment file to the created deployment version. This will create a revision and starts a build.

In [ ]:
upload_response = api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    file='example_deployment_package.zip'
)
print(upload_response)

Check if deployment is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'queued'
while status != 'success' and status != 'failed':
    build_status = api.builds_get(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME,
        version=DEPLOYMENT_VERSION,
        build_id=upload_response.build
    )
    status = build_status.status
    print("{:15s}".format(status), end='\r')
    sleep(1)
print(status)

The version is now available.

In [ ]:
api.deployment_versions_get(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION
).status

## Creating a request
Make sure the deployment is in 'available' state before performing deployment requests.

The first request is usually slow because of a cold start. A second request (performed within the `maximum_idle_time`, which was set to 30 minutes in this tutorial) will be much faster.

<div class="alert alert-block alert-info">
    
<b>Batch deployments:</b>
For deployments with <code>deployment_mode='batch'</code>, it's not possible to create a <i>direct</i> request. You should create a <i>batch</i> (asynchronous) request instead: use <code>batch_deployment_requests_create</code> instead of <code>deployment_requests_create</code>:<br/>

<pre><code python>data = {'input': 123}
request_batch = api.batch_deployment_requests_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=[data]
)
print(request_batch)
</code></pre>

<br/>Please, note that it is only possible to retrieve the results later if you used <code>request_retention_mode='full'</code> upon deployment version creation.<br/>

<pre><code python>request_result = api.deployment_requests_get(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    request_id=request_batch[0].id
)
print(request_result)
</code></pre>
</div>

In [ ]:
data = {'input': 123}
request_1 = api.deployment_version_requests_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    data=data
)
print(request_1)

Result:

In [ ]:
print(request_1.result)

It's possible to make a request to the *default* deployment version. The first created version automatically becomes the *default* version, and can be set to another version later. 

In [ ]:
data = {'input': 123}
request_2 = api.deployment_requests_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=data
)
print(request_2)

Result:

In [ ]:
print(request_2.result)

***

# Pipelines

Multiple deployments can be chained using a *pipeline*. In this tutorial, we will chain the deployment created above with a new deployment. Just for demo purposes, we will use the same deployment package (the same code) for both deployments in the pipeline. Therefore, the pipeline will contain two modelling steps: each step will multiply the input by a random number.

## Create another deployment and version
Normally you would like to chain different deployments. However, just for demo purposes, we will use the same deployment package as used above.

Create the deployment and version

In [ ]:
DEPLOYMENT_NAME2 = f"{DEPLOYMENT_NAME}-2"
deployment_template2 = deployment_template
deployment_template2.name = DEPLOYMENT_NAME2

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template2
)

api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2,
    data=version_template
)

Upload the zip

In [ ]:
upload_response2 = api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2,
    version=DEPLOYMENT_VERSION,
    file='example_deployment_package.zip'
)
print(upload_response2)

Wait for the deployment to be ready

In [ ]:
from time import sleep
status = 'queued'
while status != 'success' and status != 'failed':    
    build_status = api.builds_get(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME2,
        version=DEPLOYMENT_VERSION,
        build_id=upload_response2.build
    )
    status = build_status.status
    print("{:15s}".format(status), end='\r')
    sleep(1)
print(status)

## Create a pipeline and version

In [ ]:
PIPELINE_NAME = f"{DEPLOYMENT_NAME}-pipeline"
PIPELINE_VERSION = DEPLOYMENT_VERSION

In [ ]:
pipeline_template = ubiops.PipelineCreate(
    name=PIPELINE_NAME,
    description='A simple pipeline that multiplies the input float two times by a random number.',
    input_type='structured',
    output_type='structured',
    input_fields=[{'name': 'input', 'data_type': 'double'}],
    output_fields=[{'name': 'output', 'data_type': 'double'}]
)

api.pipelines_create(project_name=PROJECT_NAME, data=pipeline_template)

<div class="alert alert-block alert-info">
    
<b>Request Retention Mode:</b>
Use <code>request_retention_mode='metadata'</code> or <code>request_retention_mode='full'</code> to store all direct and batch requests to the pipeline version. Only then you will be able to retrieve them later. In this demo, we don't need to store them.
</div>

In [ ]:
pipeline_template = ubiops.PipelineVersionCreate(
    version=PIPELINE_VERSION,
    request_retention_mode='none'  # We don't need to store the requests in this demo
)

api.pipeline_versions_create(project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, data=pipeline_template)

## Add deployments as objects to pipeline version

In [ ]:
object_template = ubiops.PipelineVersionObjectCreate(
    name=DEPLOYMENT_NAME,
    reference_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION
)
api.pipeline_version_objects_create(
    project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, version=PIPELINE_VERSION, data=object_template
)

It's also possible to refer to the *default* deployment version. Let's do that for the second object.

In [ ]:
object_template2 = ubiops.PipelineVersionObjectCreate(
    name=DEPLOYMENT_NAME2,
    reference_name=DEPLOYMENT_NAME2
)
api.pipeline_version_objects_create(
    project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, version=PIPELINE_VERSION, data=object_template2
)

## Connect the objects

start -> deployment1

In [ ]:
attachment_template = ubiops.AttachmentsCreate(
    destination_name=DEPLOYMENT_NAME,
    sources=[
        ubiops.AttachmentSourcesCreate(
            source_name='pipeline_start',
            mapping=[ubiops.AttachmentFieldsCreate(source_field_name='input', destination_field_name='input')]
        )
    ]
)

api.pipeline_version_object_attachments_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME, 
    version=PIPELINE_VERSION,
    data=attachment_template
)

deployment1 -> deployment2

In [ ]:
attachment_template = ubiops.AttachmentsCreate(
    destination_name=DEPLOYMENT_NAME2,
    sources=[
        ubiops.AttachmentSourcesCreate(
            source_name=DEPLOYMENT_NAME,
            mapping=[ubiops.AttachmentFieldsCreate(source_field_name='output', destination_field_name='input')]
        )
    ]
)

api.pipeline_version_object_attachments_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME, 
    version=PIPELINE_VERSION,
    data=attachment_template
)

deployment2 -> end

In [ ]:
attachment_template = ubiops.AttachmentsCreate(
    destination_name='pipeline_end',
    sources=[
        ubiops.AttachmentSourcesCreate(
            source_name=DEPLOYMENT_NAME2,
            mapping=[ubiops.AttachmentFieldsCreate(source_field_name='output', destination_field_name='output')]
        )
    ]
)

api.pipeline_version_object_attachments_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME, 
    version=PIPELINE_VERSION,
    data=attachment_template
)

## Creating a request

<div class="alert alert-block alert-info">
    
<b>Batch deployments:</b>
If your pipeline contains one or more deployments with <code>deployment_mode='batch'</code>, it's not possible to create a pipeline <i>direct</i> request. You should create a pipeline <i>batch</i> request instead: use <code>batch_pipeline_requests_create</code> instead of <code>pipeline_requests_create</code>.<br/>

<pre><code python>data = {'input': 123}
pipeline_batch = api.batch_pipeline_requests_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    data=[data]
)
print(pipeline_batch)
</code></pre>

<br/>Please, note that it is only possible to retrieve the results later if you use <code>request_retention_mode='full'</code>.<br/>

<pre><code python>pipeline_result = api.pipeline_requests_get(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    request_id=pipeline_batch[0].id
)
print(pipeline_result)
</code></pre>
</div>

In [ ]:
data = {'input': 123}
pipeline_result = api.pipeline_version_requests_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    version=PIPELINE_VERSION,
    data=data
)
print(pipeline_result)

It's also possible to make a request to the *default* pipeline version. Like for deployments, the first created pipeline version automatically becomes the *default* version, and can be set to another version later.

In [ ]:
data = {'input': 123}
pipeline_result = api.pipeline_requests_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    data=data
)
print(pipeline_result)

## Cleanup
Delete created deployment.

In [ ]:
api.pipelines_delete(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME
)

In [ ]:
api.deployments_delete(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME
)
api.deployments_delete(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2
)

## Close connection
Close the connection with the UbiOps API client.

In [ ]:
client.close()